In [29]:
import csv
import random
import numpy as np
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

In [42]:
# model = SentenceTransformer('llmrails/ember-v1')
model = SentenceTransformer('WhereIsAI/UAE-Large-V1')

No sentence-transformers model found with name WhereIsAI/UAE-Large-V1. Creating a new one with MEAN pooling.


config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [43]:
sentences = [
    "This is an example sentence",
    "Each sentence is converted"
]
embeddings = model.encode(sentences)
print(cos_sim(embeddings[0], embeddings[1]))

tensor([[0.7438]])


In [44]:
# load the meddra terms
fh = open('data/meddra_llt_pt_map.txt')
reader = csv.reader(fh, delimiter='|')
header = next(reader)

meddra_llt_terms = set()
meddra_pt_terms = set()

for row in reader:
    meddra_llt_terms.add(row[1].lower())
    meddra_pt_terms.add(row[4].lower())

fh.close()

meddra_terms = sorted(meddra_llt_terms | meddra_pt_terms)
len(meddra_terms)

73302

In [45]:
sampled_meddra_terms = random.sample(meddra_terms, 100) + ["Hyperglycaemia"]
sampled_embeddings = model.encode(sampled_meddra_terms)

In [46]:
query = "high blood glucose"
query_embedding = model.encode(query)
query_embedding

array([-0.21122518, -0.05385017, -0.02172772, ...,  0.0987644 ,
        0.18727395,  0.04142275], dtype=float32)

In [47]:
cos_sims = list()
for i in range(sampled_embeddings.shape[0]):
    cos_sims.append(float(cos_sim(query_embedding, sampled_embeddings[i])))

In [48]:
sampled_meddra_terms[np.argmax(cos_sims)], np.max(cos_sims)

('Hyperglycaemia', 0.8582732081413269)